In [41]:
from langchain.callbacks import get_openai_callback
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
    ChatPromptTemplate,
)


In [42]:
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=1e-2)


In [43]:
system_message = """
Your job is to determine the payout for an injury, if possible. \
Use these rules to determine the payout:

1. For an injury involving the lower body, the payout is 500 SEK.
2. The payout is 300 SEK, if the injury is on the left arm.
3. A right arm injury does not qualify for a payout, i.e., the payout is 0 SEK.
4. If the person recieved a blow to the head, the payout is 1000 SEK.

Be advised! An injury might not be covered by the rules. \
In that case, you should let them know that you first have to talk with your collegaues before proceeding. \
In case of multiple injuries, add a total payout in your reply. \
It is very important that you always refer to the rule number when replying with the payout for an injury.
""".strip()
system_message_prompt = SystemMessagePromptTemplate.from_template(system_message)

print(system_message_prompt.format().content)


Your job is to determine the payout for an injury, if possible. Use these rules to determine the payout:

1. For an injury involving the lower body, the payout is 500 SEK.
2. The payout is 300 SEK, if the injury is on the left arm.
3. A right arm injury does not qualify for a payout, i.e., the payout is 0 SEK.
4. If the person recieved a blow to the head, the payout is 1000 SEK.

Be advised! An injury might not be covered by the rules. In that case, you should let them know that you first have to talk with your collegaues before proceeding. In case of multiple injuries, add a total payout in your reply. It is very important that you always refer to the rule number when replying with the payout for an injury.


In [44]:
example_human_message = """
Yesterday, I fell during my bicycle ride to Grandma's and broke my leg.
""".strip()
example_human_message_prompt = SystemMessagePromptTemplate.from_template(
    example_human_message,
    additional_kwargs={"name": "example_user"},
)

example_ai_message = """
I am sorry to hear that. \
According to rule #1, you will receive a payout of 500 SEK for the injury to your leg.
""".strip()
example_ai_message_prompt = SystemMessagePromptTemplate.from_template(
    example_ai_message,
    additional_kwargs={"name": "example_assistant"},
)

print(example_human_message_prompt.format().content)
print(example_ai_message_prompt.format().content)


Yesterday, I fell during my bicycle ride to Grandma's and broke my leg.
I am sorry to hear that. According to rule #1, you will receive a payout of 500 SEK for the injury to your leg.


In [45]:
human_message = "{injury}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_message)

chat_messages = [
    system_message_prompt,
    example_human_message_prompt,
    example_ai_message_prompt,
    human_message_prompt,
]

chat_prompt = ChatPromptTemplate.from_messages(chat_messages)
chain = LLMChain(llm=chat, prompt=chat_prompt)


In [46]:
def decide_payout(injury: str) -> str:
    with get_openai_callback() as cb:
        response = chain.run(injury)
        print(f"Total Tokens: {cb.total_tokens}")
        print(f"Total Cost (USD): ${cb.total_cost}")
        return response


In [47]:
decide_payout("A brick fell on my head during construction work.")

Total Tokens: 289
Total Cost (USD): $0.0005780000000000001


'I am sorry to hear that. According to rule #4, you will receive a payout of 1000 SEK for the injury to your head.'

In [48]:
decide_payout("A dinosaur ate my right arm during the shooting of Jurrasic Park X.")


Total Tokens: 320
Total Cost (USD): $0.00064


"I'm sorry to hear that. According to rule #3, a right arm injury does not qualify for a payout. However, since this is an unusual case, I will have to consult with my colleagues before proceeding. Can you please provide me with more details about the incident?"

In [49]:
decide_payout("A dinosaur ate my left arm during the shooting of Jurrasic Park X.")


Total Tokens: 295
Total Cost (USD): $0.00059


"I'm sorry to hear that. According to rule #2, you will receive a payout of 300 SEK for the injury to your left arm."

In [50]:
decide_payout("I was hit by a car.")


Total Tokens: 273
Total Cost (USD): $0.000546


'I am sorry to hear that. Can you please specify the location and type of injury?'

In [51]:
decide_payout("My heart was broken by love.")


Total Tokens: 301
Total Cost (USD): $0.000602


"I'm sorry to hear that, but I'm afraid that this type of injury is not covered by our rules. However, if you have any other injuries, please let me know and I will do my best to assist you."

In [52]:
decide_payout("I hurt both my left arm and my right knee.")


Total Tokens: 320
Total Cost (USD): $0.00064


'According to rule #1, you will receive a payout of 500 SEK for the injury to your knee. According to rule #2, you will receive a payout of 300 SEK for the injury to your left arm. Therefore, the total payout for your injuries is 800 SEK.'

In [53]:
decide_payout("Jag har brutit benet. Aj aj vad ont det gjorde och det här skriver jag ju helt på svenska 😃")


Total Tokens: 316
Total Cost (USD): $0.0006320000000000001


'Jag beklagar att du har brutit benet. Enligt regel #1 kommer du att få en utbetalning på 500 SEK för skadan på ditt ben.'